# Graph Data Science for Supply Chain
## Multi-Route Optimization for Supply Chain and Logistics
![Neo4j version](https://img.shields.io/badge/Neo4j->=4.4.9-brightgreen)
![GDS version](https://img.shields.io/badge/GDS-2.3-brightgreen)

This notebook demonstrates how [Neo4j Graph Data Science](https://neo4j.com/docs/graph-data-science/current/algorithms/) can be applied to optimize multi-input and multi-output business process.

This is a continuation of `part1-route-optimization-single-paths.ipynb`.  We will be using the same Cargo 2000 case study dataset of freight forwarding shipment routes.

It is worth noting that while we are using an air cargo freight forwarding dataset in this example, the same techniques can be applied to other types of logistics, including maritime and trucking services as well as other types of supply chain domains including manufacturing, inventory management, and bill of materials.

## Prerequisites
- The Cargo 2000 case study dataset must be loaded into Neo4j. You can do so by running [this notebook](https://github.com/neo4j-product-examples/ds-supply-chain-use-cases/blob/main/modeling-eda-visualization-c2k/transform-and-load.ipynb). It should take no more than a few minutes to complete.


In [3]:
from graphdatascience import GraphDataScience

In [4]:
# Use Neo4j URI and credentials according to your setup
gds = GraphDataScience('neo4j://localhost', auth=('neo4j', 'examplePassword'))

## Dataset

For a sample dataset we will use the “Cargo 2000” transport and logistics case study [[1]](#1). Cargo 2000 (re-branded as Cargo iQ in 2016) is an initiative of the International Air Transport Association (IATA) that aims to deliver a new quality management system for the air cargo industry.

The below figure shows a model of the business processes covered in the IATA case study. It represents the business processes of a freight forwarding company, in which up to three smaller shipments from suppliers are consolidated and then shipped together to customers. The business process is structured into incoming and outgoing transport legs, with the overall objective that freight is delivered to customers in a timely manner.  You can find out more about the business model in [this blog](https://neo4j.com/developer-blog/supply-chain-neo4j-gds-bloom/) where we explored the dataset in Neo4j Bloom or from the [original data source]( https://s-cube-network.eu/c2k/).

<img src="img/logistics-diagram.png" alt="summary" width="1000"/>

## References
<a id="1">[1]</a> A. Metzger, P. Leitner, D. Ivanovic, E. Schmieders, R. Franklin, M. Carro, S. Dustdar, and K. Pohl, “ Comparing and combining predictive business process monitoring techniques,” IEEE Trans. on Systems Man Cybernetics: Systems, 2015.omparing and combining predictive business process monitoring techniques,” IEEE Trans. on Systems Man Cybernetics: Systems, 2015.

## Understanding Historic Shipment Routes
We transform the above data into the graph data model depicted below.

<img src="img/c2k-schema.png" alt="summary" width="1000"/>




Individual shipments consist of one or multiple incoming legs that merge into a single outgoing leg for delivery.  Each leg, incoming and outgoing, has the potential for multiple connecting flights.  For reference, below is an example of what a single shipment looks like in the graph with three incoming legs originating from Davidburgh, Paulchester, and Moodytown, and outgoing leg going to Davisfort.


<img src="img/paths-of-single-shipments.png" alt="summary" width="1000"/>

For more details on understanding historic shipments please see `part1-route-optimization-single-paths.ipynb`.

## Multi-Path Route Optimization with Steiner Trees

In `part1-route-optimization-single-paths.ipynb` we showed how shortest path algorithms can be used to recommend routes between an origin and a destination. While helpful, this doesn’t directly optimize the problem for our freight forwarding network.

In this dataset the goal of a shipment is to get freight from multiple origins to a single destination. As such, the real business problem is not in optimizing single paths, but optimizing a process that takes freight from *multiple* origins and sends them to a single destination. If we measure cost by the effective time of the business processes, then we can combine freight from multiple origins over different segments or legs to reduce cost. i.e. we can reduce total time in transit by combining freight.

Solving this problem by brute-force is computationally expensive and scales poorly, becoming infeasible for large graphs. As a result we need some sort of approximate solution.

We will explore some approaches below using an example where we have 4 shipments going to Sandersshire from the airports
1. Bradleymouth
2. Moodytown
3. Richardberg, and
4. Wandaborough


In [5]:
def drop_graph_by_name(graph_name: str):
    if gds.graph.exists(graph_name).loc['exists']:
        gds.graph.get(graph_name).drop()

To optimize this problem we can turn to the [Minimum Directed Steiner Tree Algorithm](https://neo4j.com/docs/graph-data-science/current/algorithms/directed-steiner-tree/) which estimates optimal sub-networks in a graph. Specifically, it uses approximation methods to find a tree connecting a source node to a set of target nodes with the goal of making the sum of relationship weights in the tree as small as possible.

Directed Steiner trees have a variety of applications including:
- Supply chain and logistics operations
- Infrastructure planning
- Cyber & network routing
- Microchip design
- Autonomous maintenance
- Variants of traveling salesman problems (TSP)

Let's consider an example where we have 4 shipments going to Sandersshire from the airports
1. Bradleymouth
2. Moodytown
3. Richardberg, and
4. Wandaborough

In [6]:
source_airport_names =  ['Bradleymouth', 'Moodytown', 'Richardberg', 'Wandaborough']
target_airport_name = 'Sandersshire'

To run the minimum directed Steiner tree algorithm, we start by projecting the graph.  This time we must reverse the relationship orientation to match the Steiner tree algorithm, since we are looking to go from multiple origins to a single destination.

We will use the average effective time of the business processes as the relationship weight to minimize

In [7]:
drop_graph_by_name('proj')
g, _ = gds.graph.project.cypher('proj',
'''
    MATCH(n)
    WHERE n:EntryPoint
       OR n:DepartureWarehouse OR n:DeparturePoint OR n:ArrivalWarehouse OR n:TransferPoint OR n:Destination
    RETURN id(n) as id, labels(n) as labels
''',
'''
    MATCH(n0)-[r:RECEPTION|DEPARTURE|TRANSPORT|DELIVERY]->(n1)
    RETURN id(n0) AS target, id(n1) AS source, type(r) AS type, avg(r.effectiveMinutes) AS averageEffectiveMinutes
'''
)
_

nodeQuery            MATCH(n)\n    WHERE n:EntryPoint\n       OR n:...
relationshipQuery    MATCH(n0)-[r:RECEPTION|DEPARTURE|TRANSPORT|DEL...
graphName                                                         proj
nodeCount                                                         1422
relationshipCount                                                 2024
projectMillis                                                      132
Name: 0, dtype: object

Once that is done we can go ahead and run the algorithm in write mode. This will return summary statistics to the Python client and write the resulting tree back to the database so we can visualize in Bloom/Workspace later.

In [8]:
origin_node_ids = [gds.find_node_id(['EntryPoint'], {'name': source_airport_name}) for source_airport_name in source_airport_names]
destination_node_id = gds.find_node_id(['Destination'], {'name': target_airport_name})

In [9]:
stats = gds.beta.steinerTree.write(g, sourceNode=destination_node_id, targetNodes = origin_node_ids,
                                   relationshipWeightProperty='averageEffectiveMinutes', writeRelationshipType='MDST_SOLUTION_PATH', writeProperty='weight')
stats

writeMillis                                                                367
relationshipsWritten                                                        15
effectiveNodeCount                                                          16
effectiveTargetNodesCount                                                    4
totalWeight                                                       14069.456562
preProcessingMillis                                                          0
computeMillis                                                               35
configuration                {'writeConcurrency': 4, 'sourceNode': 1641, 'a...
Name: 0, dtype: object

In [10]:
print(f'Steiner tree solution - total average effective minutes: {round(stats["totalWeight"],2):,}')

Steiner tree solution - total average effective minutes: 14,069.46


The solution can be visualized in Bloom/Workspace using a [graph pattern search](https://neo4j.com/docs/bloom-user-guide/current/bloom-tutorial/search-phrases-advanced/), you want to look for nodes connected with the `MDST_SOLUTION_PATH` relationship

<img src="img/steiner-tree-solution.png" alt="summary" width="1000"/>

## Cleanup

In [ ]:
# remove solution paths and graph projection
gds.run_cypher('MATCH ()-[r:MDST_SOLUTION_PATH]->() DELETE r')
g.drop()